# ETL Project
---

### **Description**

#### In this ETL project i will use pyspark as an ETL tool to extract data from postgres database (company) and make transformation using dataframe operations then load it in DWH (Company_DWH) with star schema

In [68]:
from pyspark.sql import SparkSession

In [69]:
spark = SparkSession.builder \
    .appName("PostgresTest") \
    .config("spark.driver.extraClassPath", './postgresql-42.3.9.jar') \
    .config("spark.executor.extraClassPath", "./postgresql-42.3.9.jar") \
    .getOrCreate()

In [ ]:
# Here we established connection to postgres server
jdbc_url = "jdbc:postgresql://127.0.0.1:5432/company"
properties = {
    "user": "postgres",
    "password": "P@ssw0rd",
    "driver": "org.postgresql.Driver"
}

In [ ]:
# for each table in my database i will creat dataframe
customers_df = spark.read.jdbc(url=jdbc_url, table="public.customers", properties=properties)

In [5]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- created_at: timestamp (nullable = true)



In [ ]:
# We will see many nulls and dublicates values. We will clean it
customers_df.show(10, truncate=False)

+-----------+---------+----------------------+------------+--------------+--------------------------+
|customer_id|full_name|email                 |phone       |address       |created_at                |
+-----------+---------+----------------------+------------+--------------+--------------------------+
|1          |John Doe |john.doe93@example.com|NULL        |123 Elm St.   |2024-12-13 16:07:58.781834|
|2          |John Doe |john.doe55@example.com|123-456-4281|123 Elm Street|2024-06-26 16:07:58.781834|
|3          |John Doe |john.doe87@example.com|123-456-6859|123 Elm Street|2024-05-03 16:07:58.781834|
|4          |John Doe |john.doe36@example.com|123-456-5522|123 Elm St.   |2024-09-06 16:07:58.781834|
|5          |John Doe |NULL                  |NULL        |123 Elm Street|2025-02-08 16:07:58.781834|
|6          |John Doe |john.doe7@example.com |123-456-6407|123 Elm Street|2024-12-28 16:07:58.781834|
|7          |John Doe |NULL                  |NULL        |123 Elm St.   |2024-11-

In [8]:
orders_df = spark.read.jdbc(url=jdbc_url, table="public.orders", properties=properties)

In [9]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- amount: decimal(38,18) (nullable = true)
 |-- status: string (nullable = true)



In [10]:
orders_df.show(10, truncate=False)

+--------+-----------+-----------------------------+----------------------+---------+
|order_id|customer_id|order_date                   |amount                |status   |
+--------+-----------+-----------------------------+----------------------+---------+
|1       |171        |2024-10-02 16:07:58.781834+00|830.190000000000000000|Completed|
|2       |876        |2024-09-29 16:07:58.781834+00|743.880000000000000000|Completed|
|3       |362        |2024-12-28 16:07:58.781834+00|762.310000000000000000|Completed|
|4       |603        |2024-10-17 16:07:58.781834+00|NULL                  |Completed|
|5       |578        |2024-12-12 16:07:58.781834+00|375.830000000000000000|Completed|
|6       |139        |2024-12-14 16:07:58.781834+00|83.020000000000000000 |completed|
|7       |12         |2024-09-30 16:07:58.781834+00|182.670000000000000000|completed|
|8       |713        |2024-06-07 16:07:58.781834+00|484.240000000000000000|Completed|
|9       |938        |2025-02-08 16:07:58.781834+00|NU

In [ ]:
payments_df = spark.read.jdbc(url=jdbc_url, table="public.payments", properties=properties)

In [11]:
payments_df.printSchema()

root
 |-- payment_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- payment_date: string (nullable = true)
 |-- amount: decimal(38,18) (nullable = true)
 |-- method: string (nullable = true)



In [ ]:
payments_df.show(10, truncate=False)

+----------+--------+-----------------------------+----------------------+-----------+
|payment_id|order_id|payment_date                 |amount                |method     |
+----------+--------+-----------------------------+----------------------+-----------+
|1         |27      |2024-03-30 16:07:58.781834+00|990.000000000000000000|Credit Card|
|2         |72      |2024-09-10 16:07:58.781834+00|832.240000000000000000|Credit Card|
|3         |100     |2025-02-22 16:07:58.781834+00|NULL                  |Credit Card|
|4         |229     |2024-12-09 16:07:58.781834+00|569.350000000000000000|credit card|
|5         |982     |2024-04-21 16:07:58.781834+00|210.770000000000000000|credit card|
|6         |705     |2024-05-29 16:07:58.781834+00|52.050000000000000000 |Credit Card|
|7         |870     |2024-05-04 16:07:58.781834+00|578.620000000000000000|Credit Card|
|8         |530     |2024-10-29 16:07:58.781834+00|846.130000000000000000|credit card|
|9         |22      |2024-08-18 16:07:58.78

#### Now lets do some cleaning on these dataframes 

In [ ]:
# Remove Dublicates
customers_df = customers_df.dropDuplicates(["email", "phone"])

In [15]:
# Null Handling
customers_df = customers_df.fillna({"email": "unknown@example.com", "phone": "000-000-0000", "address": "Unknown"})

In [16]:
# Make data consistent with same format
from pyspark.sql.functions import lower, col
customers_df = customers_df.withColumn("email", lower(col("email")))

In [17]:
# fix date column
from pyspark.sql.functions import to_date

orders_df = orders_df.withColumn("order_date", to_date(col("order_date"), "yyyy-MM-dd"))

In [18]:
orders_df = orders_df.dropna(subset=["order_date"])

In [19]:
orders_df = orders_df.withColumn("status", lower(col("status")))

In [20]:
payments_df = payments_df.withColumn("method", lower(col("method")))

## Now we will make DWH
#### i will create another database to not use the OLTP one

#### with star schema design
#### - Dim_Customer
#### - Dim_Date
#### - Dim_Payment_Method
#### - Fact_Orders

In [33]:
from pyspark.sql.functions import col, year, month, dayofmonth, date_format

In [34]:
dim_customer = customers_df.select("customer_id", "full_name", "email", "phone", "address").dropDuplicates()

In [35]:
dim_customer.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = false)
 |-- phone: string (nullable = false)
 |-- address: string (nullable = false)



In [36]:
dim_date = orders_df.select(col("order_date").alias("full_date")) \
    .withColumn("year", year(col("full_date"))) \
    .withColumn("month", month(col("full_date"))) \
    .withColumn("day", dayofmonth(col("full_date"))) \
    .withColumn("weekday", date_format(col("full_date"), "EEEE")) \
    .dropDuplicates()

In [37]:
dim_date.printSchema()

root
 |-- full_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)



In [40]:
dim_payment_method = payments_df.select(col("method").alias("payment_method")).dropDuplicates()

In [41]:
dim_payment_method.printSchema()

root
 |-- payment_method: string (nullable = true)



In [58]:
fact_orders = orders_df.join(payments_df.select("order_id", "method"), ["order_id"], "left")

In [59]:
fact_orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- amount: decimal(38,18) (nullable = true)
 |-- status: string (nullable = true)
 |-- method: string (nullable = true)



In [60]:
fact_orders = fact_orders.join(dim_customer, ["customer_id"], "left") \
                         .join(dim_date, fact_orders["order_date"] == dim_date["full_date"], "left") \
                         .drop("full_date") \
                         .join(dim_payment_method, fact_orders["method"] == dim_payment_method["payment_method"], "left") \
                         .drop("method")

In [61]:
from pyspark.sql.functions import col, date_format, monotonically_increasing_id

fact_orders = fact_orders.withColumn("date_id", date_format(col("order_date"), "yyyyMMdd").cast("integer"))

In [62]:
from pyspark.sql.functions import when


fact_orders = fact_orders.withColumn(
    "method_id",
    when(col("payment_method") == "Credit Card", 1)
    .when(col("payment_method") == "PayPal", 2)
    .when(col("payment_method") == "Bank Transfer", 3)
    .otherwise(0)  
)


In [63]:
fact_orders.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- amount: decimal(38,18) (nullable = true)
 |-- status: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- method_id: integer (nullable = false)



In [64]:
from pyspark.sql.functions import year, month, dayofmonth

fact_orders = fact_orders.withColumn("date_id", 
    (year("order_date") * 10000) + (month("order_date") * 100) + dayofmonth("order_date")
)

In [65]:
fact_orders = fact_orders.select("order_id", "customer_id", "date_id", "amount", "method_id")

In [66]:
fact_orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- amount: decimal(38,18) (nullable = true)
 |-- method_id: integer (nullable = false)



### Load data to DWH

In [ ]:
def write_to_postgres(df, table_name):
    df.write \
      .format("jdbc") \
      .option("url", "jdbc:postgresql://127.0.0.1:5432/company_DWH") \
      .option("dbtable", table_name) \
      .option("user", "postgres") \
      .option("password", "P@ssw0rd") \
      .mode("append") \
      .save()

write_to_postgres(dim_customer, "Dim_Customer")
write_to_postgres(dim_date, "Dim_Date")
write_to_postgres(dim_payment_method, "Dim_Payment_Method")
write_to_postgres(fact_orders, "Fact_Orders")